<a href="https://colab.research.google.com/github/Alecia113/NLP-Emotional-analysis/blob/main/AE5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  

import pickle #必要的
training_data = pickle.load(open("training_data.pkl","rb")) #必要的
testing_data = pickle.load(open("testing_data.pkl","rb")) #必要的

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data))) #8000
print("Size of testing dataset: {0}".format(len(testing_data))) #2000
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data[0][0], training_data[0][1]))
print("------------------------------------")
#LABEL: neg / SENTENCE: hopeless for tmr :(

------------------------------------
Size of training dataset: 8000
Size of testing dataset: 2000
------------------------------------
------------------------------------
Sample Data
LABEL: neg / SENTENCE: hopeless for tmr :(
------------------------------------


In [2]:
import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#分集合

train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]

test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))] #neg pos

#做预处理 变小写
import pprint
text_train = [s.lower() for s in train_data]
text_test = [s.lower() for s in test_data]

# 分词
import pprint
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()


train_t=[]  #完全每句话的分词的集合。train_t[0][0]每个单词
for s in text_train:
  text_train = tknzr.tokenize(s)                         #每句话分别作了分词 text_train[0]第一句话的分词
  train_t.append(text_train)  
        #:(    :-(


test_t=[]     #2000
for w in text_test:
  text_test = tknzr.tokenize(w)
  test_t.append(text_test)

#删
#theprincesszooz but i see what youre going at   
#yes  subjective pain may not be real
#  but that does not make it less painful

import re
def remove(x):
  t = []
  for i in range(len(x)):
    t_sub = []        #是直接用空列表代替了
    for j in range(len(x[i])):
      if len(x[i][j])==0:
        continue
      if x[i][j] == " ":
        continue 
      else:
        x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
        if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
          x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
        if len(x[i][j]) == 0:
          continue 
        else:
          t_sub.append(x[i][j]) 
    t.append(t_sub) 

  return t

text_train = remove(train_t)
text_test = remove(test_t)

##停用词  
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words()
sww = sw.words()

text_train_le=[]    #8000
for tokens in text_train:
    filtered_sentence1 = [w for w in tokens if not w in stop_words]
    text_train_le.append(filtered_sentence1)


text_test_le=[]   #2000
for tokens in text_test:
  filtered_sentence2 = [w for w in tokens if not w in stop_words]
  text_test_le.append(filtered_sentence2)

#去掉复数 stemmer 词根不需要这部； 如果用就需要在neg和pos需要stemmer操作

from nltk.stem.porter import *
stemmer = PorterStemmer()
#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
train_stem = []
for i in range(len(text_train_le)):
  singles1 = []
  for plural in text_train_le[i]:
    singles1.append(stemmer.stem(plural))
  train_stem.append(singles1)

print(train_stem[:5])

test_stem = []
for i in range(len(text_test_le)):
  singles2 = []
  for plural in text_test_le[i]:
    singles2.append(stemmer.stem(plural))
  test_stem.append(singles2)

print(test_stem[:5])



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[['hopeless', 'tmr', ':('], ['everyth', 'kid', 'section', 'ikea', 'cute', 'shame', "i'm", 'nearli', 'month', ':('], ['hegelbon', 'heart', 'slide', 'wast', 'basket', ':('], ['ketchburn', 'hate', 'japanes', 'call', 'bani', ':(', ':('], ['dang', 'start', 'next', 'week', 'work', ':(']]
[['andreamarysmith', 'help', '...', 'stop', 'cri', ':('], ['realyys_', 'otl', 'nevermind', ':(', 'least', 'got', 'jeon'], ['soon', 'tweet', 'plant', 'claw', 'thigh', 'traction', 'zoom', 'away', ':('], ['luketothestar', 'damnit', ':('], ['klm', 'use', 'pri', 'pv', '...', 'wish', 'could', 'reliv', 'day', 'becom', 'nyc', 'pv', 'buy', 'way', 'commun', 'nyc', 'usa', 'klm', ':(']]


W@B


In [3]:
from gensim.models import Word2Vec
wv_cbow_model = Word2Vec(sentences= train_stem + test_stem, size=100, window=5, min_count=2, workers=2, sg=0)


In [4]:
#训练
wv_cbow_model.save("cbow.model")
#训练 ； 存 取；训练。
cbow = Word2Vec.load("./cbow.model")
cbow.train(sentences= train_stem + test_stem ,total_examples=wv_cbow_model.corpus_count, epochs=cbow.iter)
#举个例子，已经训练好的word2vec词表有[“我”,“爱”,“你”]三个词，给定一个新语料[“我”,“喜”,“欢”,“你”]，新语料只会调整原有词中的“我”“你”的词向量表示，而不会将“喜”“欢”添加到词库中。

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


(236282, 368710)

In [5]:
#否定词"ISO-8859-1"==UTF-8
from google.colab import drive
drive.mount('/content/drive') #必须要挂载
not_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding="ISO-8859-1").readlines()]
#处理文件呢。
j = -1
for i in not_words:
  j +=1
  if i == '2-faced':
    break
neg = not_words[j:]

#处理文件呢。
sure_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/positive-words.txt', 'r', encoding="ISO-8859-1").readlines()]

j = -1
for i in sure_words:
  j +=1
  if i == 'a+':
    break

pos = sure_words[j:]

#neg pos 词表
#要对他们做stem因为训练集做了stem


Mounted at /content/drive


In [6]:
#也要对他做stem #太久了。简直了 至少10-20分钟，太久了。
neg_stem = []
for i in range(len(neg)):
  singles3 = []
  for plural in neg:
    singles3.append(stemmer.stem(plural))
  neg_stem.append(singles3)

print(neg_stem[:5])


pos_stem = []
for i in range(len(pos)):
  singles4 = []
  for plural in pos:
    singles4.append(stemmer.stem(plural))
  pos_stem.append(singles4)

print(pos_stem[:5])



[['2-face', '2-face', 'abnorm', 'abolish', 'abomin', 'abomin', 'abomin', 'abomin', 'abort', 'abort', 'abort', 'abrad', 'abras', 'abrupt', 'abruptli', 'abscond', 'absenc', 'absent-mind', 'absente', 'absurd', 'absurd', 'absurdli', 'absurd', 'abus', 'abus', 'abus', 'abus', 'abysm', 'abysm', 'abyss', 'accident', 'accost', 'accurs', 'accus', 'accus', 'accus', 'accus', 'accus', 'accusingli', 'acerb', 'acerb', 'acerb', 'ach', 'ach', 'ach', 'achey', 'ach', 'acrid', 'acridli', 'acrid', 'acrimoni', 'acrimoni', 'acrimoni', 'adam', 'adamantli', 'addict', 'addict', 'addict', 'addict', 'admonish', 'admonish', 'admonishingli', 'admonish', 'admonit', 'adulter', 'adulter', 'adulter', 'adulteri', 'adversari', 'adversari', 'advers', 'advers', 'afflict', 'afflict', 'afflict', 'affront', 'afraid', 'aggrav', 'aggrav', 'aggrav', 'aggress', 'aggress', 'aggress', 'aggressor', 'aggriev', 'aggriev', 'aggriv', 'aghast', 'agoni', 'agon', 'agon', 'agonizingli', 'agoni', 'aground', 'ail', 'ail', 'ailment', 'aimless'

In [7]:
#只需要先让他们对应起来。您要检查每个单词是在正还是负词典中not_exist（0），negative（1）和positive（2）
#一维句子，二维单词
#是要吧一句话变成【012组成的形式】
#现在就是一一对应的形式了。一句话;这个是train_stem【 as,ada,daff】 match_embedding【120】同一个索引在两个列表中 
train_match= []
for i in range(len(train_stem)):
  match = []
  for j in range(len(train_stem[i])):
    if train_stem[i][j] in neg_stem[0]:
      match.append(1)   #neg 1
    elif train_stem[i][j] in pos_stem[0]:
      match.append(2) #pos 2
    else:
      match.append(0)
  train_match.append(match)
print(train_match[:5])
print(train_stem[:5])
#只有stem好用

[[1, 0, 0], [0, 0, 0, 0, 2, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 0]]
[['hopeless', 'tmr', ':('], ['everyth', 'kid', 'section', 'ikea', 'cute', 'shame', "i'm", 'nearli', 'month', ':('], ['hegelbon', 'heart', 'slide', 'wast', 'basket', ':('], ['ketchburn', 'hate', 'japanes', 'call', 'bani', ':(', ':('], ['dang', 'start', 'next', 'week', 'work', ':(']]


#处理OOV

#Bi-directional RNN Sequence Model (3 marks)

Many-to-One (N to 1) Sequence model

In [8]:
#Apply/Import Word and Lexicon Embedding as an input:
#You are to concatenate the trained word embedding and lexicon embedding, and apply to the sequence model

In [9]:
'''
我当时min_count设置成3了，
所以place出现的次数是2被抛弃了，
后来我用了一个统计词频的函数大概是collection库里的counter函数，
把统计的词频小于三的词全部跳过了
DF： 文本在文件中出现的次数
':(': 3544,
 'hopeless': 2,
 'tmr': 4,
 'cute': 58,
 'everyth': 25,
 "i'm": 380,
 'ikea': 1,
 'kid': 28,
'''

'\n我当时min_count设置成3了，\n所以place出现的次数是2被抛弃了，\n后来我用了一个统计词频的函数大概是collection库里的counter函数，\n把统计的词频小于三的词全部跳过了\nDF： 文本在文件中出现的次数\n\':(\': 3544,\n \'hopeless\': 2,\n \'tmr\': 4,\n \'cute\': 58,\n \'everyth\': 25,\n "i\'m": 380,\n \'ikea\': 1,\n \'kid\': 28,\n'

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import math
import numpy as np


#DF

In [ ]:
DF

In [15]:
train_stem[0]

['hopeless', 'tmr', ':(']

In [16]:
match[0]

1

In [17]:
len(new_embedding)  #2

101

In [18]:
new_embedding[0]  #13

0.1919565498828888

In [19]:
len(word_vec)

100

In [20]:
len(new_embedding)  #101维度了

101

In [21]:
#说明
len(input)  #8000 句子
len(input[0])  #3  一句话中多少词
len(input[0][0]) #每个词的词向量 101

101

In [22]:
#要的 防止OOV的问题
DF = {}

for tokensized_doc in train_stem:
    # get each unique word in the doc - and count the number of occurrences in the document
    for term in np.unique(tokensized_doc):
        try:
            DF[term] +=1
        except:
            DF[term] =1
#DF 整个统计的词频。 保留DF大于2的词。 if DF[term]>=2 我就重新添加。
input = []
for w in range(len(train_stem)):  #8000
  num = -1
  new = []
  for t in train_stem[w]: #一句话 #t就是这个单词
    match = []  #目前这个就是第一句话的match
    num += 1
    if DF[t] >= 2:
      word_vec = wv_cbow_model[t].tolist()
      match.append(train_match[w][num]) #很有可能错哦
    
      new_embedding = word_vec + match   #sent_embedding
      new.append(new_embedding)
  input.append(new)  # input == train_embedding
#print(input[0][0]) #input 8000一句话 3一个词 101 vec+0、1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


**build training sequence model** Bi-RNN or Bi-LSTM or Bi-GRU) [N to 1]

In [23]:
#主要是为了得到n_class
'''
已经串联完了，要给它扔进序列模型。
你要创建序列模型了。Build training sequence model
把你的输入扔到这里Bi-directional RNN-based (Bi-RNN or Bi-LSTM or Bi-GRU) 【N to1】(N: word, One: Sentiment - Positive or Negative)
自己决定超参
'''
#n你后面可是12
import numpy as np

from sklearn.preprocessing import LabelEncoder

unique_labels = np.unique(train_label)

lEnc = LabelEncoder()
# Please encode the labels (Do NOT add new lines of code in this section)
# Hint: Try to understand the difference between fit_transform and transform
label_train_encoded = lEnc.fit_transform(train_label)
label_test_encoded = lEnc.fit_transform(test_label)

n_class = len(unique_labels)  #主要是为了得到n_class  #n_class = np.unique(unique_labels).shape[0] #2相当于
print(unique_labels)
print(lEnc.transform(unique_labels))
print(n_class)

['neg' 'pos']
[0 1]
2


In [24]:
len(input)

8000

In [27]:
doc_length_list[3662]

22

In [26]:
#seq_length 参数怎么来的
doc_length_list = []  #会得到8000句话
maxlength = 0
for doc in train_stem:
    doc_length_list.append(len(doc))#每句话多少个分词

max = doc_length_list.index(max(doc_length_list))#python 寻找list中最大元素对应的索引方法
doc_length_list[max] #maxlength

22

In [28]:
#超参的设置没什么大问题了
import numpy as np

seq_length = doc_length_list[max] #（22）应该是我规定的最大序列长度512那种  maxlength 增加减少那种 #一句话最长多少个词
n_input = len(input[0][0])  #101 无所谓的反正都是101维度
n_class = len(unique_labels) #2

n_hidden = 200
batch_size = 256
total_epoch = 20
learning_rate = 1e-3

In [37]:
#apply to the sequence model
'''
那是不是还要把它长度补齐。
'''
doc_length_list
np.array(input).shape#要先把list 边array  #8000,3,101,因为没补齐。


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


(8000,)

In [41]:
len(input[0])

3

**padding and encoding** 填充和编码

In [75]:
input = []
for w in range(len(train_stem)):  #8000
  num = -1
  new = []
  for t in train_stem[w]: #一句话 #t就是这个单词
    match = []  #目前这个就是第一句话的match
    num += 1
    if DF[t] >= 2:
      word_vec = wv_cbow_model[t].tolist()
      match.append(train_match[w][num]) #很有可能错哦
    
      new_embedding = word_vec + match   #sent_embedding
      new.append(new_embedding)
  input.append(new)  # input == train_embedding
#print(input[0][0]) #input 8000一句话 3一个词 101 vec+0、1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [88]:
len(input[0]) #8000 3 101  sent = input[0] 一句话

3

In [ ]:
sen = input[0][0]  #3
sen   #期中的单词

In [ ]:
encoded = sen + zero
encoded

In [161]:
len(input[m])

12

In [168]:
#[【adfaf0]0 0 0 0 0] #第一句话

22

In [200]:
np.array(sent_encoded).shape

(8000, 22, 101)

In [ ]:
torch.tensor(input[0])

In [ ]:
seq_length - len(input[m])

In [198]:
len(zero[8])

101

In [201]:
#都变成一样长22 每个里面都是101
sent_encoded = []
for m in range(len(input)): # m 1-8000 #训练集
  #n = 0
  encoded = [] 
#for sen in input[m]: #把每个单词拿出来
  #n = 0            #取你合成的每个句子 第一个句子是3  i o y p o [0] [1]
  zero = []
  if len(input[m]) < seq_length:
    zero = [len(input[0][0])*[0]] * (seq_length - len(input[m]))#补充到22 最长 # 不单单加0； 还是需要变成101维度
    encoded = input[m] + zero
  else:
    encoded = input[m]
  sent_encoded.append(encoded) #要重新添加。 句子已经全变成22了 

sent_encoded = np.array(sent_encoded)
  #n +=1
#sent_encoded.append(encoded)
  
  #elif len(sent) > seq_length:  #貌似不可能，除非你改
    #sent = sent[:seq_length]
  #sent_encoded.append(sent[n] + zero) #要重新添加。

#return sent_encoded
'''
他是让train_pad_encoded == sent_encoded #8000
np.array(input).shape # (8000,22,101)
np.array(sent_encoded).shape # (8000,22,101)
array([[[ 1.08281560e-02, -3.45292059e-03,  4.73330216e-03, ...,
          6.01722207e-03, -1.59898191e-03,  1.00000000e+00],
        [ 2.36799773e-02, -7.71258911e-03,  1.43275419e-02, ...,
          1.16952946e-02, -6.32952899e-03,  0.00000000e+00],
        [ 7.00934649e-01, -3.57719600e-01,  4.85395104e-01, ...,
          3.58456910e-01, -1.29591584e-01,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
'''

'\n他是让train_pad_encoded == sent_encoded #8000\nnp.array(input).shape # (8000,22,101)\nnp.array(sent_encoded).shape # (8000,22,101)\n'

list

In [ ]:
np.array(input).shape # (8000,22,101)

**build training sequence model**

In [ ]:
cbow

In [ ]:
input[0][0][0]

In [ ]:
input_batch

In [234]:
import torch.nn as nn
import torch.nn.functional as F

class Bi_RNN_Model(nn.Module):
    def __init__(self):
        super(Bi_RNN_Model, self).__init__()
        # set the bidirectional to True
        self.rnn = nn.RNN(n_input, n_hidden, batch_first =True, bidirectional=True)
        self.linear = nn.Linear(2*n_hidden,3) #n_class = 3 
        #https://stackoverflow.com/questions/60259836/cnn-indexerror-target-2-is-out-of-bounds 虽然是01分类但是，pytorch 要进行012 
    def forward(self, x):        
        x, h_n = self.rnn(x)
        # concat the last hidden state from two direction
        hidden_out = torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        output = self.linear(hidden_out)
        return output


from sklearn.metrics import accuracy_score
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
model = Bi_RNN_Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#Train model : **Loss+epochs**

In [205]:
sent_encoded.shape

(8000, 22, 101)

In [206]:
type(sent_encoded)

numpy.ndarray

In [209]:
len(train_label)

8000

In [220]:
tag

7999

In [227]:
np.array(label_train)

array([1, 1, 1, ..., 2, 2, 2])

In [ ]:
outputs

In [235]:
#处理
train_label = np.array(train_label)
#neg 1 pos2
label_train = []
for tag in train_label:
  if tag == 'neg':
    label_train.append(1)
  else:
    label_train.append(2)
#print(label_train)
#print(len(label_train))
label_train = np.array(label_train)

for epoch in range(total_epoch):
    train_loss = 0
    for ind in range(0,sent_encoded.shape[0],batch_size):
        input_batch = sent_encoded[ind:min(ind+batch_size, sent_encoded.shape[0])] #256,22,101
        target_batch = label_train[ind:min(ind+batch_size, sent_encoded.shape[0])]  #256
        input_batch_torch = torch.from_numpy(input_batch).float().to(device)  #256,22,101
        target_batch_torch = torch.from_numpy(target_batch).view(-1).to(device) #256

        model.train()
        optimizer.zero_grad()
        outputs = model(input_batch_torch)   #13,512,25 #256，22，101    #256 2
        loss = criterion(outputs, target_batch_torch) #256 2    256
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    print('Epoch: %d, train loss: %.5f'%(epoch + 1, train_loss))

print('Finished Training')

## Prediction
model.eval()
outputs = model(torch.from_numpy(test_embeddings).float().to(device)) 
predicted = torch.argmax(outputs, 1)

# classification_report builds a text report showing the main classification metrics
# The returned report includes the 'weighted avg f1' we want (refer to the sample output)
# More details can be found from: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
from sklearn.metrics import classification_report
print(classification_report(test_label, predicted.cpu().numpy(),digits=4))

Epoch: 1, train loss: 73.41993
Epoch: 2, train loss: 30.61723
Epoch: 3, train loss: 33.90181
Epoch: 4, train loss: 21.71714
Epoch: 5, train loss: 25.87432
Epoch: 6, train loss: 21.45001
Epoch: 7, train loss: 15.01114
Epoch: 8, train loss: 42.05673
Epoch: 9, train loss: 30.13689
Epoch: 10, train loss: 20.44400
Epoch: 11, train loss: 41.73692
Epoch: 12, train loss: 23.39709
Epoch: 13, train loss: 62.21067
Epoch: 14, train loss: 78.69200
Epoch: 15, train loss: 61.89145
Epoch: 16, train loss: 59.98755
Epoch: 17, train loss: 56.09535
Epoch: 18, train loss: 52.13857
Epoch: 19, train loss: 50.20482
Epoch: 20, train loss: 48.38515
Finished Training


NameError: ignored

#TEST

In [ ]:
#我不在这里变torch我到用的地方去变torch
#目标变torch
#lexicon = torch.tensor(0,dtype=torch.long)
'''
inputs = []
for i in range(len(input)):
  tor_input =[]
  for j in range(len(input[i])):
    
    #tor_input.append(torch.tensor(input[i][j]))  # 101
print(len(inputs))
'''

In [ ]:
#type(input[0][0][0]) #list list float

In [ ]:
    #if DF[term] >=  2:  #只要最小值之下的我都不要了。

'''
    if DF[term] >=  2:  #只要最小值之下的我都不要了。
      word_vec = wv_cbow_model['term'].tolist()
      for i in range(len(train_stem)):
        new = []
        for j in range(len(train_stem[i])):
          match = []
          #if train_match[i][j] != 0:
          word_vec = wv_cbow_model['term'].tolist()
          match.append(train_match[i][j])
          new_embedding = word_vec + match
          new.append(new_embedding)
        print(new)
'''


In [ ]:
'''
import numpy as np

for i in range(len(train_stem)):
  for j in range(len(train_stem[i])):
    match = []
    if train_match[i][j] != 0:
      word_vec = wv_cbow_model[train_stem[i][j]].tolist()
      match.append(train_match[i][j])
      new_embedding = word_vec + match
  print(new_embedding)
  # k = torch.tensor(wv_cbow_model["train_stem[i][j]"],dtype=torch.long) 
  #   #wv_cbow_model["train_stem[i][j]"]  + match_embedding[i][j] = input
  #只有一个想法为啥会跑崩溃

'''

In [ ]:
import numpy as np
new_word = wv_cbow_model["hope"].tolist()
type(new_word)

In [ ]:
match = []
match.append(train_match[0][0])
match

In [ ]:
input = new_word + match
len(input)  #101; list
#https://blog.csdn.net/roytao2/article/details/54180182?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161915629316780265461518%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161915629316780265461518&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-54180182.first_rank_v2_pc_rank_v29&utm_term=python如何将两个list合并为一个

In [ ]:
type(wv_cbow_model["hope"])

In [ ]:
word_vec = torch.tensor(wv_cbow_model["hope"])
word_vec
lexicon = torch.tensor(0)
lexicon
#torch.cat((word_vec, lexicon),1) #序列中数据是任意相同的shape的同类型tensors

In [ ]:
#cbow["hope"] #(100,) 新的调用

In [ ]:
#word_vec = torch.tensor(wv_cbow_model["hope"],dtype=torch.long)
#lexicon = torch.tensor(0,dtype=torch.long)

#input = torch.cat((word_vec, lexicon),1) 
#print(input)
